# Importing packages

In [1]:
!pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 10.8 MB/s eta 0:00:00


In [2]:
!pip install pyxdf

In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mne
import pyxdf
import glob
import os
import matplotlib.pyplot as plt
from scipy.io import loadmat
import scipy
import sklearn
# ------------------------------------------------------------------------------------------
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as lda
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import ShuffleSplit, cross_val_score, train_test_split, GridSearchCV, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC

# ------------------------------------------------------------------------------------------
from mne.decoding import CSP
from mne import Epochs, pick_types
from mne.channels import make_standard_montage
from mne.datasets import eegbci
from mne.io import concatenate_raws, read_raw_edf



In [4]:
import warnings
warnings.filterwarnings('ignore') # to ignore warnings

In [5]:
verbose = False                    # global variable to suppress output display of MNE functions
mne.set_log_level(verbose=verbose) # to suppress large info outputs

# Data Loading

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Session 2
# Anwar_data_path = "/content/drive/MyDrive/OurGPData/OurDatau/sub-Anwar/ses-S002/eeg"
# Anwar_data_path = "/content/drive/MyDrive/processed/sub-Anwar/ses-S002/eeg/Remaining Data"
# Anwar_data_path = "/content/drive/MyDrive/processed/sub-Anwar/ses-S002/all_data_s2"
# Sherif_data_path = "/content/drive/MyDrive/OurGPData/OurDatau/sub-Sherif/ses-S002/eeg"

# Session 3
# Anwar_data_path = "/content/drive/MyDrive/processed/sub-Anwar/ses-S003/s3"
# Anwar_data_path = "/content/drive/MyDrive/processed/sub-Anwar/ses-S003/s3_45"
# Anwar_data_path = "/content/drive/MyDrive/processed/sub-Anwar/ses-S003/s3_00"
# Sherif_data_path = "/content/drive/MyDrive/processed/sub-Sherif/ses-S003/eeg"

# All
# Anwar_data_path = "/content/drive/MyDrive/processed/sub-Anwar/s-00"
# Sherif_data_path = "/content/drive/MyDrive/OurGPData/all sherif"

# Magdy

Magdy_data_path = "/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg"


# folder_path = Anwar_data_path
# folder_path = Sherif_data_path
folder_path  = Magdy_data_path

In [8]:
files   = glob.glob(folder_path + '/*eeg.xdf')
len(files)     # if  return zero,then no file is loaded

80

In [9]:
files

['/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Left Grasp_run-001_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Right Grasp_run-002_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Right Release_run-003_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Baseline_run-004_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Right Grasp_run-005_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Right Release_run-006_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Left Grasp_run-007_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Baseline_run-008_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/s

In [10]:
# filtered_paths = [path for path in files if 'Right Grasp' in path or 'Right Release' in path]
filtered_paths = [path for path in files if 'Right Grasp' in path or 'Right Release' in path]
filtered_paths
# len(filtered_paths)

['/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Right Grasp_run-002_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Right Release_run-003_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Right Grasp_run-005_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Right Release_run-006_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Right Grasp_run-010_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Right Release_run-013_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Right Release_run-014_eeg.xdf',
 '/content/drive/MyDrive/OurGPData/OurData/sub-magdy/ses-S001/eeg/sub-magdy_ses-S001_task-Right Grasp_run-015_eeg.xdf',
 '/content/drive/MyDrive/OurGPDa

In [11]:
all_files = filtered_paths

# from XDF to MNE

In [12]:
# read stream from xdf
streams, header = pyxdf.load_xdf(all_files[0])
# extract data
data = streams[0]["time_series"].T
# check that it is 9 channels
# assert data.shape[0] == 9 # 9 raw EEG channels

#get channels count
ch_count = int(streams[0]["info"]["channel_count"][0])
# extract channels names
# ch_names = []
# for i in range(ch_count):
#   ch_names.append(streams[0]["info"]["desc"][0]["channels"][0]["channel"][i]["label"][0])
# extract sampling ratw
fs = float(streams[0]["info"]["nominal_srate"][0])
# create info
# info = mne.create_info(ch_names, fs, "eeg")

In [13]:
data.shape

(9, 4005)

In [14]:
# extract channels names
ch_names = []
for i in range(ch_count):
  ch_name = streams[0]["info"]["desc"][0]["channels"][0]["channel"][i]["label"][0]
  # ch_name= ch_name.split("\n")[1]
  ch_name= ch_name.split("\n")[0]
  ch_names.append(ch_name)

In [15]:
ch_names

['C3', 'CZ', 'C4', 'FCZ', 'FC3', 'CP4', 'CPZ', 'CP3', 'FC4']

In [16]:
# cleaned_channel_names = [name.replace('[', '').replace(']', '') for name in ch_names]
# cleaned_channel_names

In [17]:
crack_channel_names = ['C3', 'Cz', 'C4', 'Fz', 'F3', 'P4', 'Pz', 'P3', 'F4']

In [18]:
# info = mne.create_info(crack_channel_names, fs, "eeg")
info = mne.create_info(ch_names, fs, "eeg")

In [19]:
def extract_tasks_names(files):
  task_names = []
  for path in files:
      # Split the path to isolate the filename
      filename = path.split('/')[-1]
      # Extract the task name part
      task_name = filename.split('_')[2]
      # Replace the hyphen with a space to match the desired format
      task_name = task_name.split('-')[-1]
      # Append the task name to the list
      task_names.append(task_name)
  return task_names

In [20]:
task_names = extract_tasks_names(all_files)
print(task_names)
# event_id = {'Left_Grasp': 1, 'Right_Grasp': 2, 'Right_Release': 3, 'Baseline': 4}
event_id = {'Right Grasp': 1, 'Right Release': 2, 'Baseline': 3}
# event_id = {'Right Grasp': 1, 'Right Release': 2, 'B': 3}

# Convert the list of task names to the list of numbers
task_numbers = [event_id[task] for task in task_names]
print(task_numbers)

['Right Grasp', 'Right Release', 'Right Grasp', 'Right Release', 'Right Grasp', 'Right Release', 'Right Release', 'Right Grasp', 'Right Release', 'Right Release', 'Right Grasp', 'Right Release', 'Right Grasp', 'Right Release', 'Right Grasp', 'Right Grasp', 'Right Release', 'Right Grasp', 'Right Release', 'Right Grasp', 'Right Release', 'Right Grasp', 'Right Grasp', 'Right Grasp', 'Right Release', 'Right Release', 'Right Release', 'Right Release', 'Right Grasp', 'Right Grasp', 'Right Release', 'Right Release', 'Right Release', 'Right Grasp', 'Right Grasp', 'Right Grasp', 'Right Release', 'Right Grasp', 'Right Grasp', 'Right Release']
[1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2]


In [21]:
task_names

['Right Grasp',
 'Right Release',
 'Right Grasp',
 'Right Release',
 'Right Grasp',
 'Right Release',
 'Right Release',
 'Right Grasp',
 'Right Release',
 'Right Release',
 'Right Grasp',
 'Right Release',
 'Right Grasp',
 'Right Release',
 'Right Grasp',
 'Right Grasp',
 'Right Release',
 'Right Grasp',
 'Right Release',
 'Right Grasp',
 'Right Release',
 'Right Grasp',
 'Right Grasp',
 'Right Grasp',
 'Right Release',
 'Right Release',
 'Right Release',
 'Right Release',
 'Right Grasp',
 'Right Grasp',
 'Right Release',
 'Right Release',
 'Right Release',
 'Right Grasp',
 'Right Grasp',
 'Right Grasp',
 'Right Release',
 'Right Grasp',
 'Right Grasp',
 'Right Release']

In [22]:
# crop parametes
t_start = 1
t_end = 6
idx_start = int(fs*t_start)
idx_end = int(fs*t_end)

In [23]:
all_data=[]
cont = 0
for fname in all_files:
  streams, header = pyxdf.load_xdf(fname)
  data = streams[0]["time_series"].T
  cont = cont + 1
  print(data.shape)
  if len(data) != 0:
    print(cont)
    # print(data)
    # crop data from second 1 to 6
    cropped_data = data[:,idx_start:idx_end+1]
    if (cropped_data.shape == (9, 2005)):
      print(fname)
    all_data.append(cropped_data)

(9, 4005)
1
(9, 4005)
2
(9, 3510)
3
(9, 4515)
4
(9, 4020)
5
(9, 4980)
6
(9, 3765)
7
(9, 3750)
8
(9, 3795)
9
(9, 4020)
10
(9, 3525)
11
(9, 3810)
12
(9, 3990)
13
(9, 4245)
14
(9, 3750)
15
(9, 4245)
16
(9, 3750)
17
(9, 4020)
18
(9, 4020)
19
(9, 3510)
20
(9, 3510)
21
(9, 3255)
22
(9, 3510)
23
(9, 3255)
24
(9, 3750)
25
(9, 3990)
26
(9, 3975)
27
(9, 4020)
28
(9, 4005)
29
(9, 3750)
30
(9, 4005)
31
(9, 4020)
32
(9, 3990)
33
(9, 4005)
34
(9, 3540)
35
(9, 4275)
36
(9, 4020)
37
(9, 3765)
38
(9, 3780)
39


(9, 3510)
40


In [24]:
len(all_data)
for data in all_data:
  print(data.shape)

(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)
(9, 2501)


In [25]:
# Combine into a 3D array
combined_array = np.stack(all_data, axis=0)
print(combined_array.shape)

(40, 9, 2501)


In [26]:
eeg_data = combined_array

In [27]:
epochs = mne.EpochsArray(eeg_data, info, verbose=verbose, tmin=0)

In [28]:
# epochs.set_montage('standard_1020')
epochs.filter(1., None)
# epochs.apply_baseline(baseline=(-.250, 0)) # linear baseline correction

epochs.event_id = event_id
epochs.events[:,2] = task_numbers

In [29]:
epochs

Number of events,40
Events,Baseline: 0Right Grasp: 20Right Release: 20
Time range,0.000 – 5.000 s
Baseline,off


In [30]:
pip install --upgrade mne

In [31]:
# from mne.time_frequency import psd_welch

# def extract_features(preprocessed_data_list):
#     features_list = []

#     for raw in preprocessed_data_list:
#         # Calculate power spectral density
#         psds, freqs = psd_welch(raw, fmin=1., fmax=40., n_fft=2048)

#         # Extract features (e.g., mean power in specific frequency bands)
#         features = np.mean(psds, axis=2)
#         features_list.append(features)

#     return features_list

# features_list = extract_features(epochs)

# Extract features from the preprocessed data
# from mne.time_frequency import psd_array_welch

# def extract_features(preprocessed_data_list):
#     features_list = []

#     for raw in preprocessed_data_list:
#         # Extract data from the Raw object
#         data = raw.get_data()

#         # Calculate power spectral density
#         psds, freqs = psd_array_welch(data, sfreq=sfreq, fmin=1., fmax=40., n_fft=2048)

#         # Extract features (e.g., mean power in specific frequency bands)
#         features = np.mean(psds, axis=2)
#         features_list.append(features)

#     return features_list

# features_list = extract_features(epochs)

# import numpy as np
# from mne.time_frequency import psd_array_welch

# def extract_features(preprocessed_data_list, sfreq):
#     features_list = []

#     for raw in preprocessed_data_list:
#         # Get data and sampling frequency
#         data = raw.get_data()
#         n_channels = data.shape[0]

#         # Calculate power spectral density
#         psds, freqs = psd_array_welch(data, sfreq=sfreq, fmin=1., fmax=40., n_fft=2048, n_overlap=1024, average='mean')

#         # Extract features (e.g., mean power in specific frequency bands)
#         features = np.mean(psds, axis=1)
#         features_list.append(features)

#     return features_list

# # Example usage
# # Assuming `epochs` is a list of mne.Epochs objects
# sfreq = 100  # Replace with your actual sampling frequency
# features_list = extract_features(epochs, sfreq)

import numpy as np
from mne.time_frequency import psd_array_welch

def extract_features(preprocessed_data_list, sfreq):
    features_list = []

    for data in preprocessed_data_list:
        # Calculate power spectral density
        psds, freqs = psd_array_welch(data, sfreq=sfreq, fmin=1., fmax=40., n_fft=2048, n_overlap=1024, average='mean')

        # Extract features (e.g., mean power in specific frequency bands)
        features = np.mean(psds, axis=1)
        features_list.append(features)

    return features_list

# Example usage
# Assuming `epochs` is a list of NumPy arrays
sfreq = 100  # Replace with your actual sampling frequency
features_list = extract_features(epochs, sfreq)

# Apply CSP

In [32]:
# Define a monte-carlo cross-validation generator (reduce variance):

scores = []
epochs_data_train = epochs.get_data(copy=False)
cv = ShuffleSplit(20, test_size=0.2, random_state=42)
cv_split = cv.split(epochs_data_train)

# Assemble a classifier
lda = LinearDiscriminantAnalysis()
csp = CSP(n_components=2, reg=None, log=True, norm_trace=False)

# Use scikit-learn Pipeline with cross_val_score function
clf = Pipeline([("CSP", csp), ("LDA", lda)])
scores = cross_val_score(clf, epochs_data_train, task_numbers, cv=cv, n_jobs=None)

# Printing the results
class_balance = np.mean(task_numbers == task_numbers[0])
class_balance = max(class_balance, 1.0 - class_balance)
print(f"Classification accuracy: {np.mean(scores)} / Chance level: {class_balance}")

# plot CSP patterns estimated on full data for visualization
# csp.fit_transform(epochs_data_train, task_numbers)

# csp.plot_patterns(epochs.info, ch_type="eeg", units="Patterns (AU)", size=1.5)

Classification accuracy: 0.65 / Chance level: 1.0


# Apply CSP 2

In [33]:
data = epochs.get_data()
# data = data[:,:,512+256:-256] # # from 0.5s to 4.5s

# Assuming data shape is (n_epochs, n_channels, n_samples)
# For example, data = np.random.rand(100, 64, 2048) # 100 epochs, 64 channels, 2048 samples

# Adjust the slicing based on the sampling rate
sampling_rate = 500  # Hz
start_time_sec = 0
end_time_sec = 6

# Convert to sample indices
start_sample = int(start_time_sec * sampling_rate)
end_sample = int(end_time_sec * sampling_rate)

# Slice the data accordingly
data = data[:, :, start_sample:end_sample]

# Verify the shape of the sliced data
print(data.shape)  # Should reflect the new number of samples per epoch

# Calculate and print the duration in seconds
n_samples = data.shape[2]
duration_sec = n_samples / sampling_rate
print(f"The data covers {duration_sec} seconds per epoch.")

labels = epochs.events[:, -1]

# Print the data and labels for verification
print("Data shape:", data.shape)
print("Labels shape:", labels.shape)
print("Labels:", labels)



(40, 9, 2501)
The data covers 5.002 seconds per epoch.
Data shape: (40, 9, 2501)
Labels shape: (40,)
Labels: [1 2 1 2 1 2 2 1 2 2 1 2 1 2 1 1 2 1 2 1 2 1 1 1 2 2 2 2 1 1 2 2 2 1 1 1 2
 1 1 2]


In [34]:
class Custom_Segmented_CSP(TransformerMixin, BaseEstimator):
    """
    Apply CSP individually to each window and then merge their features
    Expects data in the format (trials, channels, eeg_data)
    individually apply CSP on each band and then concatenate to give output of the form (trials, csp_filtered_data)
    By Default Applies CSP on a single window [0.0,4.0] sec
    Note: This funciton expects arrays/lists as input for t_start and t_end
    """
    def __init__(self, n_components=4, t_start=[0.0], t_end=[6.0], fs=500):
        self.n_components = n_components           # csp components to retain
        self.Csp = []                              # would carry list of CSP's applied individually to each window
        self.t_start = t_start
        self.t_end = t_end
        self.fs = 500
        self.start_idxs = (np.array(self.t_start)*self.fs).astype(int)
        self.end_idxs =   (np.array(self.t_end)*self.fs).astype(int)
        self.num_windows = 0

    def fit(self, x, y):
        self.num_windows = len(self.start_idxs)
        self.Csp = [CSP(n_components=self.n_components) for _ in range(self.num_windows)]
        for i in range(self.num_windows):
            x_seg = x[:,:,self.start_idxs[i]:self.end_idxs[i]]
            self.Csp[i].fit(x_seg, y)
        return self

    def transform(self, x, y=None):
        dummy_array = []
        for i in range(self.num_windows):
            x_seg = x[:,:,self.start_idxs[i]:self.end_idxs[i]]
            dummy_array.append(self.Csp[i].transform(x_seg))
        return np.concatenate(dummy_array, axis=-1)

In [38]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Using all channels, custom CSP on 11 overlapping windows of segmented data
x_trainVal, x_test, y_trainVal, y_test = train_test_split(data, labels.ravel(), test_size=0.2, shuffle=False, random_state=0)
# x_trainVal, x_test, y_trainVal, y_test = train_test_split(data, labels.ravel(), shuffle=True, stratify=labels, random_state=0)

t_start = np.arange(0, 1.6, 0.5)
length_window = 2.5
t_end = t_start + length_window
i = 4
x_train = x_trainVal
y_train = y_trainVal

# Define a custom scorer if needed (replace 'scorer' with your actual scoring function)
scorer = 'accuracy'

print('*'*10, 'Classification Scores Comparison with default Parameters', '*'*10)
print('KNN           : ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i, t_start, t_end), KNeighborsClassifier()), x_train, y_train, cv=cv, scoring=scorer)))
print('Log-Regression: ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i, t_start, t_end), LogisticRegression(max_iter=1000)), x_train, y_train, cv=cv, scoring=scorer)))
print('Linear SVM    : ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i, t_start, t_end), LinearSVC(random_state=0)), x_train, y_train, cv=cv, scoring=scorer)))
print('Kernel SVM    : ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i, t_start, t_end), SVC(gamma='scale')), x_train, y_train, cv=cv, scoring=scorer)))
print('LDA           : ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i, t_start, t_end), LDA()), x_train, y_train, cv=cv, scoring=scorer)))
print('Decision Tree : ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i, t_start, t_end), DecisionTreeClassifier()), x_train, y_train, cv=cv, scoring=scorer)))
print('Random Forest : ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i, t_start, t_end), RandomForestClassifier()), x_train, y_train, cv=cv, scoring=scorer)))


********** Classification Scores Comparison with default Parameters **********
KNN           :  0.37857142857142856
Log-Regression:  0.42142857142857143
Linear SVM    :  0.46428571428571425
Kernel SVM    :  0.36428571428571427
LDA           :  0.55
Decision Tree :  0.41428571428571426
Random Forest :  0.3928571428571429


# Grid Search

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
# Define the parameter grid
param_grid = {
    'C': [0.1, 1, 10, 100],
    'gamma': [1, 0.1, 0.01, 0.001],
    'kernel': ['linear', 'rbf', 'poly']
}

# Initialize the SVM classifier
svm = SVC()

# Initialize the GridSearchCV object
grid_search = GridSearchCV(svm, param_grid, refit=True, verbose=2, cv=5)

# Fit the model
grid_search.fit(X, y)

# Print the best parameters and estimator
print("Best parameters found: ", grid_search.best_params_)
print("Best estimator: ", grid_search.best_estimator_)

In [ ]:
# Define the parameter grid
param_grid_mlp = {
    'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}

# Initialize the MLP classifier
mlp = MLPClassifier(max_iter=100)

# Initialize the GridSearchCV object
grid_search_mlp = GridSearchCV(mlp, param_grid_mlp, refit=True, verbose=2, cv=5)

# Fit the model
grid_search_mlp.fit(X, y)

# Print the best parameters and estimator
print("Best parameters for MLP found: ", grid_search_mlp.best_params_)
print("Best MLP estimator: ", grid_search_mlp.best_estimator_)

In [ ]:
# Define the parameter grid
param_grid_lda = {
    'solver': ['svd', 'lsqr', 'eigen'],
    'shrinkage': [None, 'auto', 0.1, 0.5, 1.0]  # Note: 'shrinkage' is only applicable for 'lsqr' and 'eigen' solvers
}

# Initialize the LDA classifier
lda = LinearDiscriminantAnalysis()

# Initialize the GridSearchCV object
grid_search_lda = GridSearchCV(lda, param_grid_lda, refit=True, verbose=2, cv=5)

# Fit the model
grid_search_lda.fit(X, y)

# Print the best parameters and estimator
print("Best parameters for LDA found: ", grid_search_lda.best_params_)
print("Best LDA estimator: ", grid_search_lda.best_estimator_)

In [ ]:
# Define the parameter grid
param_grid_linear_svc = {
    'C': [0.1, 1, 10, 100],
    'max_iter': [1000, 2000, 3000]
}

# Initialize the LinearSVC classifier
linear_svc = LinearSVC()

# Initialize the GridSearchCV object
grid_search_linear_svc = GridSearchCV(linear_svc, param_grid_linear_svc, refit=True, verbose=2, cv=5)

# Fit the model
grid_search_linear_svc.fit(X, y)

# Print the best parameters and estimator
print("Best parameters for LinearSVC found: ", grid_search_linear_svc.best_params_)
print("Best LinearSVC estimator: ", grid_search_linear_svc.best_estimator_)

In [ ]:
# Define the parameter grid
param_grid_lr = {
    'C': [0.1, 1, 10, 100],
    'solver': ['liblinear', 'saga', 'lbfgs']
}

# Initialize the Logistic Regression classifier
lr = LogisticRegression(max_iter=1000)

# Initialize the GridSearchCV object
grid_search_lr = GridSearchCV(lr, param_grid_lr, refit=True, verbose=2, cv=5)

# Fit the model
grid_search_lr.fit(X, y)

# Print the best parameters and estimator
print("Best parameters for Logistic Regression found: ", grid_search_lr.best_params_)
print("Best Logistic Regression estimator: ", grid_search_lr.best_estimator_)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

# Define the parameter grid
param_grid_extra_trees = {
    'n_estimators': [50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Initialize the Extra Trees Classifier
extra_trees = ExtraTreesClassifier()

# Initialize the GridSearchCV object
grid_search_extra_trees = GridSearchCV(extra_trees, param_grid_extra_trees, refit=True, verbose=2, cv=5)

# Fit the model
grid_search_extra_trees.fit(X, y)

# Print the best parameters and estimator
print("Best parameters for Extra Trees found: ", grid_search_extra_trees.best_params_)
print("Best Extra Trees estimator: ", grid_search_extra_trees.best_estimator_)

In [ ]:
# using all channels, custom csp on 5 overlapping windows of segmented data
# t_start = np.arange(0,1.6,0.6)
t_start = np.arange(0,1.6,0.375)
# t_start = np.arange(0,1.6,0.5)
length_window = 2.5
t_end = t_start + length_window
i = 4
x_train = x_trainVal
print('*'*10, 'Classification Scores Comparison with default Parameters' ,'*'*10)
print('KNN           : ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i,t_start,t_end), KNeighborsClassifier()), x_train, y_train, cv=cv, scoring=scorer)))
print('Log-Regression: ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i,t_start,t_end), LogisticRegression(max_iter=1000)), x_train, y_train, cv=cv, scoring=scorer)))
print('Linear SVM    : ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i,t_start,t_end), LinearSVC(random_state=0)), x_train, y_train, cv=cv, scoring=scorer)))
print('kernal SVM    : ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i,t_start,t_end), SVC(gamma='scale')), x_train, y_train, cv=cv, scoring=scorer)))
print('LDA           : ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i, t_start, t_end), LDA()), x_train, y_train, cv=cv, scoring=scorer)))
print('Decision Tree : ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i, t_start, t_end), DecisionTreeClassifier()), x_train, y_train, cv=cv, scoring=scorer)))
print('Random Forest : ', np.mean(cross_val_score(make_pipeline(Custom_Segmented_CSP(i, t_start, t_end), RandomForestClassifier()), x_train, y_train, cv=cv, scoring=scorer)))

********** Classification Scores Comparison with default Parameters **********
KNN           :  0.3833333333333333
Log-Regression:  0.39166666666666666
Linear SVM    :  0.36666666666666664
kernal SVM    :  0.33333333333333337
LDA           :  0.4833333333333333
Decision Tree :  0.37499999999999994
Random Forest :  0.38333333333333336


In [ ]:
print('-'*10, 'Information About Window Selection', '-'*10)
t_start = np.arange(0,1.501,0.375)
length_window = 2.5
t_end = t_start + length_window
print('Total Windows: ', len(t_start))
print('Starting Time of Windows: ', t_start)
print('Ending Time of Windows  :' ,  t_end)

---------- Information About Window Selection ----------
Total Windows:  5
Starting Time of Windows:  [0.    0.375 0.75  1.125 1.5  ]
Ending Time of Windows  : [2.5   2.875 3.25  3.625 4.   ]


In [ ]:
cv = StratifiedShuffleSplit(10, random_state=0)
csp_comps = [4]

In [ ]:
# for linear svm
param_grid_linear_svm =   { 'linearsvc__C' : np.logspace(-4, 3, 15),
                           'custom_segmented_csp__n_components': csp_comps}

# lda, auto shrinkage performs pretty well mostly
shrinkage = list(np.arange(0,1.01,0.1))
shrinkage.append('auto')
param_grid_lda = {'custom_segmented_csp__n_components': csp_comps,
                  'lineardiscriminantanalysis__shrinkage': shrinkage}

In [ ]:
grids_linear_svm_list = [GridSearchCV(make_pipeline(Custom_Segmented_CSP(t_start=t_start, t_end=t_end),
                                                    LinearSVC(random_state=0)),
                               param_grid=param_grid_linear_svm, cv=cv, scoring=scorer)
                        for _ in range(len(data))]

grids_lda_list = [GridSearchCV(make_pipeline(Custom_Segmented_CSP(t_start=t_start, t_end=t_end), LDA(solver='eigen')),
                        param_grid=param_grid_lda, cv=cv,  scoring=scorer)
                 for _ in range(len(data))]

In [ ]:
def training_function(subject_index=0):
    # this time training function trains on whole training set
    print('-'*25, 'Training for Subject:', subject_index+1, '-'*25)
    # epochs = epochs_list_train[subject_index]
    data = epochs.get_data()
    data = data[:,:,256+512:-256] # from 0.5s to 4.5s
    labels = epochs.events[:,-1]

    grids_linear_svm_list[subject_index].fit(data, labels)
    print('LinearSVM: Maximum Cross Validation Score = ', round(grids_linear_svm_list[subject_index].best_score_,3))
    grids_lda_list[subject_index].fit(data, labels)
    print('LDA      : Maximum Cross Validation Score = ', round(grids_lda_list[subject_index].best_score_,3))
    print()

In [ ]:
# for subject in range(len(data)):
#     training_function(subject)